In [19]:
# !pip install tabulate

In [31]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
from tabulate import tabulate
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

def eval(df, threshold):

    # train/test split
    X = df.copy()
    y = X.pop('zipcode_new')

    cv = KFold(n_splits=10, random_state=42, shuffle=False)
    for train_index, test_index in cv.split(X):

        X_train, X_test, y_train, y_test = (X.iloc[train_index], 
                                            X.iloc[test_index], 
                                            y.iloc[train_index], 
                                            y.iloc[test_index])

        # reshape y_train
        y_train = y_train.values.reshape((len(y_train), 1))

        # use X_train to compile Document-Term Matrix M
        corpus = X_train.street.values.tolist()

        # Transforms the data into a bag of words
        train_vocab = tf.fit(corpus)
        M = tf.transform(corpus)

        ## use X_test to generate Matrix W
        q = X_test.street.values.tolist()
        q[:5]
        W = tf.transform(q).todense()
        W[:5, :]

        ## matrix mult. to get scores
        R = M @ W.T
        R[:5, :]

        ## argmax(): get the index of the largest value in each column of R
        # Each column in R represents the multiplication of the corpus tfidf matrix with the a given vector reprsentation of a query address.
        ix = np.argmax(R, axis=0) # axis=0 > run through each column
        # flatten list of lists
        ix = [item for sublist in ix.tolist() for item in sublist]
        ix[:10]

        ## max(): get the largest value in each column of R
        scores = pd.DataFrame(R).max().values.tolist()
        scores[:5]

        ## lookup most similar Zipcode
        # * The index of the largest value in R corresponds to the row in X which is most similar to the query address. 
        # * Thus we can get the most similar Zipcode from y_train.
        # * Note that we have a prediction for each of the unknown data points.
        y_pred = pd.DataFrame(y_train).iloc[ix, 0].values
        y_pred[:5]

        X_test = X_test.copy()
        X_test['zipcode'] = y_test
        X_test['zipcode_pred'] = y_pred
        X_test['scores'] = scores
        X_test.head()

        accuracy_no_threshold.append(accuracy_score(y_true=y_test, y_pred=y_pred))
    
        # remove rows below threshold
        Xi_test = X_test[X_test.scores>threshold].copy()
        y_test = Xi_test.zipcode.values.tolist()
        y_pred = Xi_test.zipcode_pred.values.tolist()
        accuracy_with_threshold.append(accuracy_score(y_true=y_test, y_pred=y_pred))

        # % rows removed by applying threshold
        lost.append(len(X_test)-len(Xi_test))
        total = len(X_test)

        return accuracy_no_threshold, accuracy_with_threshold, lost, total

    
########
# MAIN #
########

# init tf-idf
tf = TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word')

# read file
filename = '../../data/airbnbdata/201702_listings_preproc.csv'
df = pd.read_csv(filename,
                 encoding='utf-8',
                 sep=',',
                 header=0)

# Isolate rows with known Zip
df = df[~pd.isnull(df.zipcode_new)]
# print('# known zip: {}'.format(len(df)))

# select column subset
df = df[['street', 'zipcode_new']]

# keep only the street name portion 
df['street'] = df['street'].apply(lambda x: x.split(',')[0])

# init placeholders
accuracy_no_threshold, accuracy_with_threshold, lost = [], [], []
total = 0
results = []

print('10-Fold Cross Validation')
for threshold in np.linspace(start=0, stop=50, num=10):  
    (accuracy_no_threshold, 
     accuracy_with_threshold, 
     lost, 
     total) = eval(df, threshold)
    
    results.append([np.round(threshold,0),
                   np.round(np.mean(accuracy_no_threshold), 3), 
                   np.round(np.mean(accuracy_with_threshold), 3),
                   np.round(np.mean(lost)/total, 3)])
print(tabulate(results, headers=["Threshold", "Accuracy wo/thr", "Accuracy w/thr", "% Rows removed by thr"]))


10-Fold Cross Validation
  Threshold    Accuracy wo/thr    Accuracy w/thr    % Rows removed by thr
-----------  -----------------  ----------------  -----------------------
          0              0.743             0.748                    0.006
          6              0.743             0.748                    0.006
         11              0.743             0.748                    0.006
         17              0.743             0.752                    0.016
         22              0.743             0.76                     0.029
         28              0.743             0.766                    0.039
         33              0.743             0.77                     0.048
         39              0.743             0.775                    0.06
         44              0.743             0.78                     0.073
         50              0.743             0.783                    0.09
